In [2]:
from pydantic import BaseSettings
from pathlib import Path

class TestSettings(BaseSettings):
    """Define the configuration model for the TAI API service."""

    test: Path = Path("/test")


settings = TestSettings()
print(str(settings.test))
print(settings.dict())


ns-1737.awsdns-25.co.uk.
ns-488.awsdns-61.com.
ns-1129.awsdns-13.org.
ns-675.awsdns-20.net.

/test
{'test': PosixPath('/test')}


In [29]:
from textwrap import dedent
BASE_SYSTEM_MESSAGE = dedent("""\
You are a friendly tutor named {name} that works for T.A.I. As {name}, {persona}. \
You are to be a good listener and ask how you can help the student and be there for them. \
You MUST get to know them as a human being and understand their needs in order to be successful. \
To do this, you need to ask questions to understand the student as best as possible. \
If a student asks for help, you should NOT give the student answers or solve the problem for them. \
Instead, you should help them understand the material and guide them to the answer step by step. \
Each step you send should be in it's own message and not all in the same message. \
For example, if the student asks for help, you could ask them what they have tried so far and suggest what they should try next. \
You should progressively give more information to the student until they understand the material and not give them all in one message. \
The student has requested that you use responses with a technical level of a {technical_level} to help the understand the material. \
Remember, you should explain things in a way that a {technical_level} would understand. \
Most importantly, you are not to give the student answers even if they ask for them, however, you can give them hints.\
""")

print(BASE_SYSTEM_MESSAGE.format(name="Finn", persona="you are extremely positive, conversational, and outgoing", technical_level="high school student"))
print(BASE_SYSTEM_MESSAGE.format(name="Kai", persona="you are straightforward and use minimal fluffy language", technical_level="expert in the field"))

You are a friendly tutor named Finn that works for T.A.I. As Finn, you are extremely positive, conversational, and outgoing. You are to be a good listener and ask how you can help and be there for them. You MUST get to know them as a human being and understand their needs in order to be successful. To do this, you need to ask questions to understand the user as best as possible. If a student asks for help, you should NOT give the student answers or solve the problem for them. Instead, you should help them understand the material and guide them to the answer step by step. Each step you send should be in it's own message and not all in the same message. For example, if the student asks for help, you could ask them what they have tried so far and suggest what they should try next. You should progressively give more information to the student until they understand the material and not give them all in one message. The user has requested that you use responses with a technical level of a hi

In [30]:
x = [1, 2, 3]
x.insert(0, 0)

In [1]:
from typing import Dict, Union
import boto3
import json
from botocore.config import Config as BotoConfig
from botocore.exceptions import ClientError
from loguru import logger


def get_secret(secret_name: str) -> Union[Dict[str, str], str]:
    logger.info(f"Retrieving secret {secret_name}")
    session = boto3.session.Session()
    boto_config = BotoConfig(
        connect_timeout=10,
        retries={
            "max_attempts": 3,
            "mode":"standard"
        }
    )
    client = session.client(
        service_name='secretsmanager',
        config=boto_config,
        region_name='us-east-1',
    )
    try:
        logger.info(f"{secret_name}")
        response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e
    secret_string = response["SecretString"]
    try:
        return json.loads(secret_string)
    except json.JSONDecodeError:
        return secret_string

In [3]:
from taiservice.api.taibackend.taitutors.llm import TaiLLM, ChatOpenAIConfig

config = ChatOpenAIConfig(api_key=get_secret("dev/tai_service/openai/api_key"))
llm = TaiLLM(config=config)
student_messages = [
    "I'm having trouble understanding the concept of limits. Can you help me out?",
    "Sure! I can explain limits to you. Let's start with the intuition behind it and then work through some examples.",
    "I'm struggling with identifying the end behavior of polynomial functions. Can you guide me through it?",
    "Definitely! I can help you understand how to determine the end behavior of polynomial functions and provide examples to illustrate the concept.",
    "I need help understanding logarithmic equations and their properties. Can you provide some guidance?",
    "Yes, of course! Logarithmic equations can be challenging, but I'll explain the properties and methods for solving them.",
    "I'm struggling with understanding the unit circle and its relationship with trigonometric functions. Can you help clarify?",
    "Absolutely! Let's dive into the unit circle together and discuss how it relates to trigonometric functions.",
    "I need assistance with matrix operations. Can you explain how to add, subtract, and multiply matrices?",
    "Sure! Matrix operations can be complicated, but I'll explain the processes step-by-step and give you some practice problems.",
    "I'm having a hard time understanding the concept of conic sections. Can you explain them to me?",
    "Definitely! Conic sections can be a bit abstract, but I'll break down each type (circle, ellipse, hyperbola, parabola) and make them more understandable.",
    "I'm struggling with the concept of logarithmic differentiation. Can you provide a step-by-step explanation?",
    "Absolutely! Logarithmic differentiation can be a bit challenging, but I'll guide you through the steps and show you examples.",
    "I need help understanding the concept of sequences and series. Can you explain them to me?",
    "Yes, of course! Sequences and series can be complex, but I'll start with the basics and gradually introduce more advanced topics.",
    "I'm having difficulty with graphing rational functions. Can you provide some guidance?",
    "Definitely! Graphing rational functions can be challenging, but I'll explain the process and provide tips to make it easier.",
    "I'm struggling with understanding the concept of exponential growth. Can you provide a detailed explanation?",
    "Yes, I can help with that! Exponential growth can be a bit tricky, but I'll break it down and give you examples to clarify the concept.",
    "I need assistance with solving limits involving indeterminate forms. Can you guide me through the steps?",
    "Sure! Solving limits with indeterminate forms can be challenging, but I'll walk you through the strategies and techniques involved.",
    "I'm having a hard time understanding the concept of one-sided limits. Can you explain them to me?",
    "Absolutely! One-sided limits can seem confusing at first, but I'll explain them in detail and provide examples to solidify the understanding.",
    "I'm struggling with determining limits at infinity. Can you provide some clarification on how to approach these problems?",
    "Definitely! Limits at infinity can be challenging, but I'll explain the methods and techniques for tackling these types of problems.",
    "I need help understanding the properties of exponential growth functions. Can you provide an overview?",
    "Yes, of course! Understanding the properties of exponential growth functions can be tricky, but I'll go through them step-by-step and provide examples.",
    "I'm having difficulty with solving exponential growth models. Can you guide me through the process?",
    "Sure! Solving exponential growth models can be complex, but I'll break down the steps for you and show you how to approach these problems.",
    "I'm struggling with understanding the concept of population growth and how it relates to exponential functions. Can you explain it in detail?",
    "Absolutely! Population growth and its connection to exponential functions can be complicated, but I'll provide a thorough explanation and examples to illustrate the concept.",
    "I need assistance with determining growth rates and decay rates in exponential functions. Can you walk me through it?",
    "Yes, of course! The process of determining growth rates and decay rates in exponential functions can be confusing, but I'll explain it in a clear and concise manner.",
    "I'm having a hard time understanding the concept of half-life in exponential decay. Can you provide a detailed explanation?",
    "Definitely! The concept of half-life in exponential decay can be challenging, but I'll break it down and give you examples to enhance your understanding.",
    "I'm having trouble understanding the concept of limits. Can you help me out?",
    "Sure! I can explain limits to you. Let's start with the intuition behind it and then work through some examples.",
    "I'm struggling with identifying the end behavior of polynomial functions. Can you guide me through it?",
    "Definitely! I can help you understand how to determine the end behavior of polynomial functions and provide examples to illustrate the concept.",
    "I need help understanding logarithmic equations and their properties. Can you provide some guidance?",
    "Yes, of course! Logarithmic equations can be challenging, but I'll explain the properties and methods for solving them.",
    "I'm struggling with understanding the unit circle and its relationship with trigonometric functions. Can you help clarify?",
    "Absolutely! Let's dive into the unit circle together and discuss how it relates to trigonometric functions.",
    "I need assistance with matrix operations. Can you explain how to add, subtract, and multiply matrices?",
    "Sure! Matrix operations can be complicated, but I'll explain the processes step-by-step and give you some practice problems.",
    "I'm having a hard time understanding the concept of conic sections. Can you explain them to me?",
    "Definitely! Conic sections can be a bit abstract, but I'll break down each type (circle, ellipse, hyperbola, parabola) and make them more understandable.",
    "I'm struggling with the concept of logarithmic differentiation. Can you provide a step-by-step explanation?",
    "Absolutely! Logarithmic differentiation can be a bit challenging, but I'll guide you through the steps and show you examples.",
    "I need help understanding the concept of sequences and series. Can you explain them to me?",
    "Yes, of course! Sequences and series can be complex, but I'll start with the basics and gradually introduce more advanced topics.",
    "I'm having difficulty with graphing rational functions. Can you provide some guidance?",
    "Definitely! Graphing rational functions can be challenging, but I'll explain the process and provide tips to make it easier.",
    "I'm struggling with understanding the concept of exponential growth. Can you provide a detailed explanation?",
    "Yes, I can help with that! Exponential growth can be a bit tricky, but I'll break it down and give you examples to clarify the concept.",
    "I need assistance with solving limits involving indeterminate forms. Can you guide me through the steps?",
    "Sure! Solving limits with indeterminate forms can be challenging, but I'll walk you through the strategies and techniques involved.",
    "I'm having a hard time understanding the concept of one-sided limits. Can you explain them to me?",
    "Absolutely! One-sided limits can seem confusing at first, but I'll explain them in detail and provide examples to solidify the understanding.",
    "I'm struggling with determining limits at infinity. Can you provide some clarification on how to approach these problems?",
    "Definitely! Limits at infinity can be challenging, but I'll explain the methods and techniques for tackling these types of problems.",
    "I need help understanding the properties of exponential growth functions. Can you provide an overview?",
    "Yes, of course! Understanding the properties of exponential growth functions can be tricky, but I'll go through them step-by-step and provide examples.",
    "I'm having difficulty with solving exponential growth models. Can you guide me through the process?",
    "Sure! Solving exponential growth models can be complex, but I'll break down the steps for you and show you how to approach these problems.",
    "I'm struggling with understanding the concept of population growth and how it relates to exponential functions. Can you explain it in detail?",
    "Absolutely! Population growth and its connection to exponential functions can be complicated, but I'll provide a thorough explanation and examples to illustrate the concept.",
    "I need assistance with determining growth rates and decay rates in exponential functions. Can you walk me through it?",
    "Yes, of course! The process of determining growth rates and decay rates in exponential functions can be confusing, but I'll explain it in a clear and concise manner.",
    "I'm having a hard time understanding the concept of half-life in exponential decay. Can you provide a detailed explanation?",
    "Definitely! The concept of half-life in exponential decay can be challenging, but I'll break it down and give you examples to enhance your understanding.",
    "I'm having trouble understanding the concept of limits. Can you help me out?",
    "Sure! I can explain limits to you. Let's start with the intuition behind it and then work through some examples.",
    "I'm struggling with identifying the end behavior of polynomial functions. Can you guide me through it?",
    "Definitely! I can help you understand how to determine the end behavior of polynomial functions and provide examples to illustrate the concept.",
    "I need help understanding logarithmic equations and their properties. Can you provide some guidance?",
    "Yes, of course! Logarithmic equations can be challenging, but I'll explain the properties and methods for solving them.",
    "I'm struggling with understanding the unit circle and its relationship with trigonometric functions. Can you help clarify?",
    "Absolutely! Let's dive into the unit circle together and discuss how it relates to trigonometric functions.",
    "I need assistance with matrix operations. Can you explain how to add, subtract, and multiply matrices?",
    "Sure! Matrix operations can be complicated, but I'll explain the processes step-by-step and give you some practice problems.",
    "I'm having a hard time understanding the concept of conic sections. Can you explain them to me?",
    "Definitely! Conic sections can be a bit abstract, but I'll break down each type (circle, ellipse, hyperbola, parabola) and make them more understandable.",
    "I'm struggling with the concept of logarithmic differentiation. Can you provide a step-by-step explanation?",
    "Absolutely! Logarithmic differentiation can be a bit challenging, but I'll guide you through the steps and show you examples.",
    "I need help understanding the concept of sequences and series. Can you explain them to me?",
    "Yes, of course! Sequences and series can be complex, but I'll start with the basics and gradually introduce more advanced topics.",
    "I'm having difficulty with graphing rational functions. Can you provide some guidance?",
    "Definitely! Graphing rational functions can be challenging, but I'll explain the process and provide tips to make it easier.",
    "I'm struggling with understanding the concept of exponential growth. Can you provide a detailed explanation?",
    "Yes, I can help with that! Exponential growth can be a bit tricky, but I'll break it down and give you examples to clarify the concept.",
    "I need assistance with solving limits involving indeterminate forms. Can you guide me through the steps?",
    "Sure! Solving limits with indeterminate forms can be challenging, but I'll walk you through the strategies and techniques involved.",
    "I'm having a hard time understanding the concept of one-sided limits. Can you explain them to me?",
    "Absolutely! One-sided limits can seem confusing at first, but I'll explain them in detail and provide examples to solidify the understanding.",
    "I'm struggling with determining limits at infinity. Can you provide some clarification on how to approach these problems?",
    "Definitely! Limits at infinity can be challenging, but I'll explain the methods and techniques for tackling these types of problems.",
    "I need help understanding the properties of exponential growth functions. Can you provide an overview?",
    "Yes, of course! Understanding the properties of exponential growth functions can be tricky, but I'll go through them step-by-step and provide examples.",
    "I'm having difficulty with solving exponential growth models. Can you guide me through the process?",
    "Sure! Solving exponential growth models can be complex, but I'll break down the steps for you and show you how to approach these problems.",
    "I'm struggling with understanding the concept of population growth and how it relates to exponential functions. Can you explain it in detail?",
    "Absolutely! Population growth and its connection to exponential functions can be complicated, but I'll provide a thorough explanation and examples to illustrate the concept.",
    "I need assistance with determining growth rates and decay rates in exponential functions. Can you walk me through it?",
    "Yes, of course! The process of determining growth rates and decay rates in exponential functions can be confusing, but I'll explain it in a clear and concise manner.",
    "I'm having a hard time understanding the concept of half-life in exponential decay. Can you provide a detailed explanation?",
    "Definitely! The concept of half-life in exponential decay can be challenging, but I'll break it down and give you examples to enhance your understanding."
]
topics = llm.summarize_student_messages(messages=student_messages, as_questions=True)
for topic in topics:
    print(topic)

2023-07-25 07:33:55.081 | INFO     | __main__:get_secret:10 - Retrieving secret dev/tai_service/openai/api_key
2023-07-25 07:33:55.246 | INFO     | __main__:get_secret:25 - dev/tai_service/openai/api_key


Can you help me understand limits?
How do I determine the end behavior of polynomial functions?
Can you explain logarithmic equations and their properties?
How does the unit circle relate to trigonometric functions?
Can you explain matrix operations (addition, subtraction, multiplication)?
What are conic sections and how do they work?
Can you provide a step-by-step explanation of logarithmic differentiation?
What are sequences and series and how do they relate?
How do I graph rational functions?
Can you clarify the concept of exponential growth?


In [13]:
# import dedent
from textwrap import dedent

def print_test():
    text = dedent(
    """
    Hi there! I'sdfiosaf\
    i
    """
    )
    return text
print(print_test())


Hi there! I'sdfiosaf    i



In [1]:
# wriete 

True


In [4]:
from typing import Optional
from pydantic import BaseModel, root_validator, ValidationError

class User(BaseModel):
    name: Optional[str]
    age: Optional[int]

    @root_validator(pre=True)
    def check_name_and_age(cls, values):
        name = values.get('name')
        age = values.get('age')
        if name is None and age is None:
            raise ValueError('Name or age must be set')
        return values

    class Config:
        validate_assignment = True

user = User(name="John", age=25)

try:
    user.age = None
    user.name = None
except ValidationError as e:
    print(e)

1 validation error for User
__root__
  Name or age must be set (type=value_error)


In [1]:
x = [1, 2, 3]

for rank, doc_id in enumerate(x, 1):
    print(rank, doc_id)

1 1
2 2
3 3


In [2]:
from langchain.document_loaders import YoutubeLoader

In [1]:
bool((False, None))

True

In [1]:
print("Problem 1\n\nNormal Axial Stress Worked Example\n\n- A steel beam measuring $10 \\mathrm{~mm}$ wide by $20 \\mathrm{~mm}$ tall by $3 \\mathrm{~m}$ long is subjected to a compressive load of 40,000 Newtons. What is the average compressive stress in a cross section of the beam normal to the forces.\n\n![](https://cdn.mathpix.com/cropped/2023_09_08_2b7618456e7fda365732g-1.jpg?height=197&width=977&top_left_y=594&top_left_x=581)\n\nPennState\n\nPennstate\nMont Alto\n\n7\n\n$$\n\\begin{gathered}\n\\sigma=\\frac{N}{A} \\\\\n\\sigma=\\frac{40,000 \\mathrm{~N}}{(.01 \\mathrm{~m})(.02 \\mathrm{~m})}=2 \\times 10^{8} \\frac{\\mathrm{N}}{\\mathrm{m}^{2}} \\\\\n\\sigma=200 \\mathrm{MPa}\n\\end{gathered}\n$$\n\nLecture 2 Page 1\n\n")

Problem 1

Normal Axial Stress Worked Example

- A steel beam measuring $10 \mathrm{~mm}$ wide by $20 \mathrm{~mm}$ tall by $3 \mathrm{~m}$ long is subjected to a compressive load of 40,000 Newtons. What is the average compressive stress in a cross section of the beam normal to the forces.

![](https://cdn.mathpix.com/cropped/2023_09_08_2b7618456e7fda365732g-1.jpg?height=197&width=977&top_left_y=594&top_left_x=581)

PennState

Pennstate
Mont Alto

7

$$
\begin{gathered}
\sigma=\frac{N}{A} \\
\sigma=\frac{40,000 \mathrm{~N}}{(.01 \mathrm{~m})(.02 \mathrm{~m})}=2 \times 10^{8} \frac{\mathrm{N}}{\mathrm{m}^{2}} \\
\sigma=200 \mathrm{MPa}
\end{gathered}
$$

Lecture 2 Page 1




In [4]:
import boto3

# Initialize the boto3 client for S3
s3 = boto3.client('s3', region_name='us-east-1')

# The name of your S3 bucket
bucket_name = 'tai-service-class-resource-cold-store-dev'

# Create a paginator to retrieve all objects
paginator = s3.get_paginator('list_objects_v2')
pages = paginator.paginate(Bucket=bucket_name)

for page in pages:
    for obj in page['Contents']:
        # Update the metadata for objects that end with '.pdf'
        if obj['Key'].endswith('.pdf'):
            s3.copy_object(Bucket=bucket_name, 
                CopySource={'Bucket': bucket_name, 'Key': obj['Key']}, 
                Key=obj['Key'], 
                ContentType='application/pdf', 
                MetadataDirective='REPLACE'
            )

KeyboardInterrupt: 

In [4]:
%pip install redis




Note: you may need to restart the kernel to use updated packages.
foo: bar
bar: None


In [52]:
from redis import RedisCluster, Redis

r = Redis(host="localhost", port=6379, decode_responses=True)
print("Connected to Redis")
print(r.ping())
r.set('foo', 'bar')
print(f"foo: {r.get('foo')}")
print(f"bar: {r.get('bar')}")

Connected to Redis
True
foo: bar
bar: None
